In [32]:
# spark UI address
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark.sparkContext.uiWebUrl

'http://jupyter-pyspark-775457d87d-8bbf2:4042'

In [62]:
spark.sql("""select max(block_height) from bronze.base_mainnet_logs""").show(truncate=False)

+-----------------+
|max(block_height)|
+-----------------+
|40872643         |
+-----------------+



In [ ]:
# !pip install pandas

In [29]:

# Bronze layer
spark.sql("""
    CREATE TABLE IF NOT EXISTS bronze.base_mainnet_logs (
        block_height BIGINT,
        job_name STRING,
        run_id STRING,
        inserted_at TIMESTAMP,
        inserted_date DATE,
        raw STRING
    ) USING iceberg
      PARTITIONED BY (inserted_date)
""")
# 👉 建好之后 不要再 DROP / 改 schema

DataFrame[]

In [ ]:
# drop table -> delete files
# spark.conf.get("spark.sql.catalog.spark_catalog.purge", "true")
# spark.sql("""drop table bronze.base_mainnet_logs""").show(truncate=False)
# spark.sql("describe table bronze.base_mainnet_logs").show()
# spark.sql("select count(1) from bronze.base_mainnet_logs").show()
# 2552926

+--------+
|count(1)|
+--------+
|58530516|
+--------+



In [ ]:

# Silver layer
spark.sql("""
CREATE TABLE IF NOT EXISTS silver.base_mainnet_logs (
    removed BOOLEAN COMMENT '是否被回滚',
    log_index BIGINT COMMENT '在区块内的索引',
    transaction_index BIGINT COMMENT '交易在区块内的索引',
    transaction_hash STRING COMMENT '交易哈希',
    block_hash STRING COMMENT '区块哈希',
    block_number BIGINT COMMENT '区块高度',
    address STRING COMMENT '合约地址（触发事件的合约）',
    data STRING COMMENT 'ABI 编码后的数据(通常是 uint256 或 bytes)',
    topics ARRAY<STRING> COMMENT '事件签名 + indexed 参数'
)
USING ICEBERG;
""")

In [39]:
spark.sql("select count(1) from bronze.base_mainnet_logs where block_height = 40869828").show(truncate=False)

+--------+
|count(1)|
+--------+
|0       |
+--------+



In [21]:
spark.sql("""
describe bronze.kafka_base_mainnet_logs.snapshots
""").toPandas()

,col_name,data_type,comment
0,committed_at,timestamp,None
1,snapshot_id,bigint,None
2,parent_id,bigint,None
3,operation,string,None
4,manifest_list,string,None
5,summary,"map<string,string>",None


In [22]:
spark.sql("""
SELECT *
FROM bronze.kafka_base_mainnet_logs.snapshots
ORDER BY committed_at DESC
LIMIT 1
""").show(truncate=False)

+-----------------------+-------------------+-------------------+---------+---------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|committed_at           |snapshot_id        |parent_id          |operation|manifest_list                                                                                                              |summary                                                                                                                                                                                    

In [ ]:
logs_sample = [{'removed': False,
  'logIndex': 97,
  'transactionIndex': 77,
  'transactionHash': 'a2f1de724043aa937adceb266e2517790cb024b2a7aa1d244db076a311cccd2f',
  'blockHash': '156fc301558435fa8f9753ab74adce06711d223c9e7999d4738a4ffb5129343d',
  'blockNumber': 24009335,
  'address': '0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913',
  'data': '0000000000000000000000000000000000000000000000000000000000005618',
  'topics': ['ddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef',
   '0000000000000000000000005965851f21dae82ea7c62f87fb7c57172e9f2add',
   '000000000000000000000000a28062bd708ce49e9311d6293def7df63f2b0816']},
 {'removed': False,
  'logIndex': 98,
  'transactionIndex': 77,
  'transactionHash': 'a2f1de724043aa937adceb266e2517790cb024b2a7aa1d244db076a311cccd2f',
  'blockHash': '156fc301558435fa8f9753ab74adce06711d223c9e7999d4738a4ffb5129343d',
  'blockNumber': 24009335,
  'address': '0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913',
  'data': '000000000000000000000000000000000000000000000000000000000145f5e2',
  'topics': ['8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925',
   '0000000000000000000000005965851f21dae82ea7c62f87fb7c57172e9f2add',
   '00000000000000000000000057df6092665eb6058de53939612413ff4b09114e']},
 {'removed': False,
  'logIndex': 99,
  'transactionIndex': 77,
  'transactionHash': 'a2f1de724043aa937adceb266e2517790cb024b2a7aa1d244db076a311cccd2f',
  'blockHash': '156fc301558435fa8f9753ab74adce06711d223c9e7999d4738a4ffb5129343d',
  'blockNumber': 24009335,
  'address': '0x6b2C0c7be2048Daa9b5527982C29f48062B34D58',
  'data': '0000000000000000000000000000000000000000000000000000000000000000',
  'topics': ['7724394874fdd8ad13292ec739b441f85c6559f10dc4141b8d4c0fa4cbf55bdb']}]

In [11]:
# query.stop() # stop the streaming

# query.status
# query.lastProgress
# query.isActive

# check all streaming jobs
spark.streams.active

# 强制停止所有 Streaming（救命用）
# for q in spark.streams.active:
#     print("Stopping:", q.name)
#     q.stop()

[]